In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime

year_window = 15

start = f"{str(datetime.now().year-year_window)}-01-01"
end = datetime.today().strftime('%Y-%m-%d')

print(start, end)

df_market = pd.read_csv("data/Market_Stock_List.csv")

2010-01-01 2025-12-05


In [10]:
df_market.describe()

,Symbol,Company,Market,Industry,Sector,Address,Zip code,Tel.,Fax,Website
count,933,933,933,933,933,933,933,933,933,933
unique,933,933,2,9,28,920,109,888,734,908
top,ZIGA,ZIGA INNOVATION PUBLIC COMPANY LIMITED,SET,Property & Construction,-,"7-8th Floor, SCB Park Plaza 1, 18 Ratchadapise...",10120,-,-,-
freq,1,1,704,209,229,3,78,10,177,18


In [11]:
df_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Symbol    933 non-null    object
 1   Company   933 non-null    object
 2   Market    933 non-null    object
 3   Industry  933 non-null    object
 4   Sector    933 non-null    object
 5   Address   933 non-null    object
 6   Zip code  933 non-null    object
 7   Tel.      933 non-null    object
 8   Fax       933 non-null    object
 9   Website   933 non-null    object
dtypes: object(10)
memory usage: 73.0+ KB


In [12]:
df_market.nunique()

Symbol      933
Company     933
Market        2
Industry      9
Sector       28
Address     920
Zip code    109
Tel.        888
Fax         734
Website     908
dtype: int64

In [13]:
df_market["Industry"] = df_market["Industry"].replace('Industrials', 'Industrial')

In [14]:
df_ind = pd.DataFrame(df_market["Industry"].unique(), columns=["unique_values"])
df_ind.rename(columns={'unique_values': 'Industry'}, inplace=True)
df_ind['Symbol'] = None
print(df_ind.head() )

                  Industry Symbol
0               Industrial   None
1               Technology   None
2        Consumer Products   None
3  Property & Construction   None
4     Agro & Food Industry   None


In [15]:
df_all_ind = df_market.groupby("Industry")["Symbol"].apply(list).reset_index()

print(df_all_ind.head() )

                  Industry                                             Symbol
0     Agro & Food Industry  [AAI, APO, APURE, ASIAN, AU, BR, BRR, BTG, CBG...
1        Consumer Products  [88TH, AFC, AJA, APCO, AURA, BGT, BIZ, BLC, BT...
2               Financials  [ACAP, AEONTS, AF, AIRA, AMANAH, ASAP, ASK, AS...
3               Industrial  [2S, ACG, ADB, AH, AJ, AKR, ALLA, ALUCON, AMC,...
4  Property & Construction  [A, A5, AIMCG, AIMIRT, AKS, ALLY, AMATA, AMATA...


In [16]:
import yfinance as yf

for i,row in df_all_ind.iterrows():
    useable = []
    for tk in row['Symbol']:
        ticker = tk+".BK"
        data = yf.download(ticker, start=start, end=end, interval="1d", auto_adjust=False, progress=False)
        if len(data) < 2:
            continue
        data = data.dropna()
        data.index = pd.to_datetime(data.index)
        if (data.index[0].year != pd.to_datetime(start).year):
            continue
        if not data.empty:
            useable.append(tk)
    df_ind.at[i,'Symbol'] = useable
    


1 Failed download:
['WIIK.BK']: Timeout('Failed to perform, curl: (28) Operation timed out after 10013 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['HL.BK']: Timeout('Failed to perform, curl: (28) Connection timed out after 10015 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['ILM.BK']: Timeout('Failed to perform, curl: (28) Operation timed out after 10001 milliseconds with 2601 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['IMH.BK']: Timeout('Failed to perform, curl: (28) Resolving timed out after 10012 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')

1 Failed download:
['IROYAL.BK']: Timeout('Failed to perform, curl: (28) Resolving timed out after 10013 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for 

In [17]:
print(df_ind.head())

                  Industry                                             Symbol
0               Industrial  [APURE, ASIAN, BR, CFRESH, CHOTI, CM, CPF, CPI...
1               Technology  [AFC, BGT, BTNC, CPH, CPL, DTCI, EFORL, FANCY,...
2        Consumer Products  [ACAP, AEONTS, AF, AMANAH, ASK, ASP, AYUD, BAY...
3  Property & Construction  [2S, AH, AJ, AKR, ALUCON, AMC, BCT, BSBM, CEN,...
4     Agro & Food Industry  [A, A5, AKS, AMATA, AP, BLAND, BROCK, BSM, CCP...


In [18]:
# df is your original DataFrame with columns "head1" and "head2" (head2 contains lists)
expanded = pd.DataFrame(df_ind['Symbol'].tolist())                 # expand lists into columns
out = pd.concat([df_ind[['Industry']].reset_index(drop=True), expanded], axis=1)
out.columns = ['Industry'] + ['Symbol'] + [''] * (expanded.shape[1] - 1)  # header: head1, head2, , , ...
out = out.fillna('')                                          # missing -> blank/None in CSV
out.to_csv('data/Industry_all.csv', index=False)

In [19]:
ls = ['^AGRO.BK', '^CONSUMP.BK', '^FINCIAL.BK', '^INDUS.BK', '^PROPCON.BK', '^RESOURC.BK', '^SERVICE.BK', '^TECH.BK']


In [20]:
all_ind = 0
no_mc = 0
no_so = 0
no_ff = 0
ls_no_mc= []
ls_no_so= []
ls_no_ff= []
for i,row in df_ind.iterrows():
    all_ind += len(row['Symbol'])
    for tk in row['Symbol']:
        try:
            now_mc = yf.Ticker(tk+'.BK').info['marketCap']
        except:
            ls_no_mc.append(tk)
            no_mc += 1

        try:
            now_so = yf.Ticker(tk+'.BK').info['sharesOutstanding']
        except:
            ls_no_so.append(tk)
            no_so += 1

        try:
            now_ff = yf.Ticker(tk+'.BK').info['floatShares']
        except:
            ls_no_ff.append(tk)
            no_ff += 1


print(all_ind)
print("NO marketCap:", no_mc, ls_no_mc)
print("NO sharesOutstanding:", no_so, ls_no_so)
print("NO free_float:", no_ff, ls_no_ff)
cant_cal = ls_no_mc + ls_no_so + ls_no_ff


432
NO marketCap: 2 ['S&J', 'WELL']
NO sharesOutstanding: 2 ['S&J', 'WELL']
NO free_float: 11 ['ACAP', 'GSTEEL', 'MIPF', 'MNIT', 'QHBREIT', 'QHOP', 'TAPAC', 'TIF1', 'TU-PF', 'SGP', 'MANRIN']


In [21]:
print(cant_cal)

['S&J', 'WELL', 'S&J', 'WELL', 'ACAP', 'GSTEEL', 'MIPF', 'MNIT', 'QHBREIT', 'QHOP', 'TAPAC', 'TIF1', 'TU-PF', 'SGP', 'MANRIN']


In [22]:
df_cut_ind = pd.DataFrame(df_market["Industry"].unique(), columns=["unique_values"])
df_cut_ind.rename(columns={'unique_values': 'Industry'}, inplace=True)
df_cut_ind['Symbol'] = None
print(df_cut_ind.head())

                  Industry Symbol
0               Industrial   None
1               Technology   None
2        Consumer Products   None
3  Property & Construction   None
4     Agro & Food Industry   None


In [23]:
# inputs:
# prices: DataFrame indexed by date (monthly), columns = tickers (close price)
# shares: Series indexed by tickers (sharesOutstanding)
# free_float: Series indexed by tickers (free-float factor, decimal)
# M = lookback months, coverage_target = 0.95

c_v = 1
K = 10

def compute_smoothed_ffmc(prices, shares, free_float, M=3):
    # prices: DataFrame monthly closes
    const = shares * free_float             # Series length N
    # print(const.head())
    ffmc = prices.mul(const, axis=1)        # DataFrame (dates x tickers)
    # rolling mean over last M rows (align to right)
    ffmc_smoothed = ffmc.rolling(window=M, min_periods=1).mean()
    return ffmc_smoothed

def select_top_k_by_coverage(ffmc_smoothed, date, coverage_target=c_v):
    ffmc_today = ffmc_smoothed.loc[date].dropna()
    total = ffmc_today.sum()
    weights = ffmc_today / total
    weights_sorted = weights.sort_values(ascending=False)
    cumsum = weights_sorted.cumsum()
    # smallest K achieving coverage_target
    topk = weights_sorted[cumsum <= coverage_target].index.tolist()
    # if none selected (coverage_target too high), take first K default:
    if len(topk) == 0:
        topk = weights_sorted.head(K).index.tolist()
    # ensure coverage >= target (add next until >=)
    while weights_sorted[topk].sum() < coverage_target and len(topk) < len(weights_sorted):
        next_idx = weights_sorted.index[len(topk)]
        topk.append(next_idx)
    if (len(topk) > K):
        topk = topk[:K]
    return topk, weights_sorted

In [24]:
cov_col = {}
for i,row in df_ind.iterrows():
    now_ind = df_ind.iloc[i,0]
    col = []
    df_now = pd.DataFrame()
    for tk in row['Symbol']:
        if tk in cant_cal:
            continue
        t = tk+'.BK'
        df_daily = yf.download(t, start=start, end=end, interval="1d", auto_adjust=False, progress=False)
        if len(df_daily) < 2:
            continue

        price_daily = df_daily["Close"].copy()

        price_daily = price_daily.dropna()
        price_daily.index = pd.to_datetime(price_daily.index)

        if (price_daily.index[0].year != pd.to_datetime(start).year):
            continue

        price_monthly = price_daily.resample('ME').last().dropna()

        price_monthly = price_monthly.rename(columns={price_monthly.columns[0]: "Value"})
        now_so = yf.Ticker(t).info['sharesOutstanding']
        now_fs = yf.Ticker(t).info['floatShares']
        now_ff = now_fs/now_so
        price_monthly['Value'] = price_monthly['Value']*now_so*now_ff
        if (df_now.empty):
            df_now = price_monthly.copy()
            continue
        df_now = df_now.add(price_monthly, fill_value=0)

    BMV = df_now.iloc[0]['Value']
    df_now['Value'] = (df_now['Value']/BMV)*100.0
    # print(df_now.head)

    fn = "data/stockdata/" + str(now_ind) + ".csv"
    df_now.to_csv(fn, index=True, date_format="%Y-%m-%d")
    
    
        

In [25]:
# from pprint import pprint
# pprint(cov_col)
# print(df_cut_ind.head())
# print(df_cut_ind.tail())

In [26]:
# sorted_items_desc = sorted(cov_col.items(), key=lambda item: item[1], reverse=True)
# pprint(sorted_items_desc)
# cut = []
# cnt = 0
# for i in sorted_items_desc:
#     if cnt < 4:
#         cnt += 1
#         cut.append(i)
# print(cut)

In [27]:
# # df is your original DataFrame with columns "head1" and "head2" (head2 contains lists)
# expanded = pd.DataFrame(df_cut_ind['Symbol'].tolist())                 # expand lists into columns
# out = pd.concat([df_cut_ind[['Industry']].reset_index(drop=True), expanded], axis=1)
# out.columns = ['Industry'] + ['Symbol'] + [''] * (expanded.shape[1] - 1)  # header: head1, head2, , , ...
# out = out.fillna('')                                          # missing -> blank/None in CSV
# out.to_csv('data/Industry_sorted.csv', index=False)